# Export Jupyter Notebooku do LaTex/u

## Úvod

Štandardný export Jupyter-Notebooku do formátu prostredia LaTex využíva knižnicu *pandoc*,ktorá konvertuje len základné konštrukcie formátovania textu.  Rozšírenie Jupyteru o plugin LaTeX *latex_envs* umožnilo doplnenie notebookov o ďaľšie konštrukty (číslovanie rovníc, lokálne odkazy a pod.), zároveň ale konvertovaný výstup v LaTeX-e vyžaduje ručné editovanie textu, naviac nie je prispôsobené na konverziu dokumentov pozostávajúcich z viacerých notebookov tvoriacich kapitoly budúceho dokumentu. Z tohoto dôvodu vznikol skript *nb2tex*, ktorý čiastočne eliminuje vyššie uvedené nedostatky.

## Rýchlonávod

* Predpokladajme, že máte vytvorené jupyter notebooky (napr. *notebook_1.ipynb* a *notebook_2.ipynb* ) v nejakom adresári napr. **Notebooky**

* Rozbalte archív **jp2tex.zip** do nejakého adresára, napr. **Konvertor**

* Vytvorte si adresár, v ktorom sa bude vytvárať dokument, napr. **Export**

* Najskôr vygenerujeme konfiguračné súbory pre budúcu konverziu, v adresári **Export** spustite

        python3 $HOME/Konvertor/jp2tex.py -c 

* V adresári sa vytvorilo niekoľko súborov s príponou **.py** a **.tex**, pomocou ktorých viete definovať titulnú stránku dokumentu, formátovanie zdrojových textov a pod. Súbory 
**chapters_path.tex** a **graphics_path.tex** sa prepisujú pri každej konverzii, ostatné konfiguračné súbry sa nemenia. V súbore config.py zadefinujeme štruktúru budúceho dokumentu, t.j. ktoré notebooky chceme exportovať a v akom poradí budú usporiadané

        source = 'Notebooky'    

        chapters = [ 
                ['notebook_1', True,  True],
                ['notebook_2', True,  True]
              ]
              

* Vlastnú konverziu spustíme v adresári **$HOME/Export** skriptom

        python3 $HOME/Konvertor/jp2tex.py

* Ak notebooky neobsahujú chyby a neznáme konštrukcie, výsledkom skriptu je vygenerovaný koreňový súbor **main.tex** a pomocné inkludované súbory, ktoré obsahujú cesty k obrázkom a jednotlivým kapitolám.  

## Konverzia pomocou skriptu nb2tex

Pretože Jupyter-Notebook sa priebežne vyvíja a aj autori rozšírení vedomí si nedostatkov priebežne svoje diela vylepšujú a dopĺňajú, nie je možné pokladť nižšie popisovaný skript za finálny a konečný produkt. Originálny export je riadený template, ktorého popis je na stránke [Customizing nbconvert](https://nbconvert.readthedocs.io/en/latest/customizing.html)

Dokument obsahuje

* Popis funkcie konverzného skriptu
* Popis usporiadania adresárovej štruktúry notebookov
* Návod na použitie a konfiguráciu konverzného skriptu
* Popis a usporiadanie prostredia LaTex-u pre vytváranie finálnych dokumentov
* Demonštračná kapitola s použitím rozšírení LaTeX\_envs

Hlavným zámerom na vytvorenie skriptu bolo vygenerovanie takého porstredia, v ktorom sa primárny dokument edituje **len** v prostredí Jupyter-Notebooku, bez potreby ďaľších alebo len minimálnych úprav v prostredí LaTeX-u.Pôvodný notebook môže obsahovať aj interaktívne časti, ktoré nie je môžné principiálne konvertovať do statického dokumentu (animácie, videá a pod.) 

Skript *nb2tex* (aktuálna verzia 1.3/190420) napísaný v Pythone ver.3.x konvertuje Jupyter-Notebook v nasledujúcich krokoch:

* Skopíruje jeden alebo viacero notebookov do samostatnej zvolenej adresárovej štruktúry. Do pôvodných zdrojových textov skript nezasahuje. Voľba súborov je definovaná v textovom konfiguračnom súbore.

* Pre skopírované notebooky spustí v neinteraktívnom móde obnovenie obsahu notebooku, skriptov, generovanie obrázkov skriptami (*matplotlib*) a iné. Po obnovení sú v lokálnej adresárovej štruktúre vygenerované obrázky, ktoré sú štandardnou súčasťou notebooku.    

* V texte notebooku **\*.ipynb** sú pomocou regulérnych výrazov upravené a nahradené niektoré problematické konštrukcie, ako sú obrázky, vyradenie častí notebooku, ktoré nie je možné konvertovať do LaTeX-u (videá, animácie ..) a úprava vertikálneho formátovania.

* Konverzia textu notebooku na LaTeX je pomocou štandardných knverzných nástrojov *nbconvert* a *pandoc*

* Z konvertovaného textu v LaTeX-e sú odstránené časti, ktoré by bránili poskladaniu viacerých dokumentov do jedného celku. Úvodná stránka a členenie dokumentu je definované v titulnom súbore LaTeX-u.

### Konverzie obrázkov

Štandardný notebook umožňuje vloženie obrázkov bez popisu (pomocou HTML tagu <img src ...>), pri ktorých je možné nastaviť šírku v px, táto informácia nie je ale konvertovaná do LaTeX-u. Na druhej strane rozšírenie LaTeX-envs umožňuje vkladanie obrázkov s popisom a veľkosťou pomocou prostredia LaTex-u *figure{}*, informáciu o veľkosti obrázku, ale interpereter notebooku toto ignoruje. 

Pre riešenie tohoto problému skript nahradí pred konverziou HTML tag konštrukciou LaTeX-u, z tohoto dôvodu je tag rozšírený o neštandardný (ignorovaný) textový parameter *scale* a HTML tag s definíciou obrázka má potom tvar (dva apostrofy v príklade reprezentujú uvodzovky)  

    <img src=''./img/map01.png'' width=500px alt="Skalárne pole" scale="0.4"/>
    
je konvertovaný na LaTex-ový popis obrázku nahradzujúci pôvodné vloženie obrázku

    \begin{figure}[h!]
    
    \centerline{\includegraphics[scale=0.4]{./img/map01.png}}
    
    \caption{Skalárne pole}

### Potlačenie nekonvertovateľných častí notebooku

Pre zabránenie chybám pri konverzie vložených animácií, videí a pod. umožňuje skript vyradiť z konverzie riadok, pred ktorým je vložený komentár s príkazom REMOVE

    <!-- REMOVE --> 
    Odstrani nasledujuci riadok po prikaze REMOVE
    
### Riadenie vertikálneho toku textu

Pomocou príkazov vložených v komentároch je možné upraviť vertikálne formátovanie textu. Príkazy sú nahradené štandardnými makrami LaTeX-u.

    <!-- SMALLSKIP -->
    <!-- MEDSKIP -->
    <!-- BIGSKIP -->
    <!-- NEWPAGE -->

## Inštalácia 

### nb2tex

Pre konverziu je použitý skript *nb2tex* pre Python ver. 3.x. Skript sa neinštaluje, používa sa ako akýkoľvek iný skrip v OS Linux. 

### LaTex

Pre správnu konverziu notebooku je potrebné mať nainštalované základné balíky LaTeX-u, fonty a podporu slovenčiny

    sudo apt-get install texlive texlive-generic-recommended texlive-latex-extra 
    sudo apt-get install texlive-lang-czechslovak
    
tieto obsahujú len základnú podmnožinu z veľkého počtu balíkov dostupných v LaTeX-e. Pre prípadné úpravy finálneho textu je vhodné mať nainštalované aj vhodné prostredie, napr. TexMaker.

### Python a Jupyter-Notebook

Okrem štandardného Jupyter-Notebooku je potrebné nainštalovať aj rozšírenie *LaTeX\_envs* a pomocné konverzné utility, knižnicu *pandoc* pre konverziu textových formátov, *nbconvert* a *jupyter\_latex\_envs* pre export notebooku  

    sudo apt-get install pandoc
    sudo apt-get install python-setuptools
    
    pip install wheel
    pip install nbconvert
    pip install jupyter_contrib_nbextensions
    
    jupyter contrib nbextension install --user
    
Podrobnejšie je inštalácia rozšírení a ich aktivovanie v prostredí Jupyter popísané v Kapitole 2. V prípade, že používate ako default verziu Python3, použitie miesto *pip* alternatívu *pip3*.

## Použitie skriptu

### Štruktúra prostredia notebooku

Adresárovú štruktúru, v ktorej vytvárame notebooky, je vhodné usporiadať v nasledujúcom tvare:

    <Source>
        |
        +--- notebook_01.ipynb      subory notebookov, jednotlive kapitoly a pod.
        +--- notebook_02.ipynb
        +--- ...
        |
        +--- <img>                  adresar so statickymi (negenerovanymi)
               |                    obrazkami pouzitymi v notebookoch
               +-- *.png            pmocou prikazov <img src ... >
             


### Parametre skriptu, konfigurácia a spustenie

Skript je potrebné spustiť s nasledujúcimi parametrami

    usage: nb2tex.py [-h] [-c] 
    
kde

    -h    optional, zobrazenie helpu
    -c    optional, vygenerovanie konfiguracnych suborov
    
Konfiguračný súbor **config.py** definuje notebooky, ktoré budú zaradené do konverzie. Formát konfiguračného súboru:

    source = 'cesta_k_notebookom'
    
    chapters = [
                 # notebook         COPY   UPDATE            
                 ['meno_notebooku', True,  True],
                 [ .... ],
               ]
    
    # vyznam flagov
    # COPY   - ak True, notebook sa bude kopirovat, inak sa pouzije lokalna kopia
    # UPDATE - ak True, vykona sa refresh celho notebooku (spustenie skriptov, 
    #          generovanie obrazkov z matplotlibu)
    #          volbu False je mozne zvolit len pri notebookoch, ktore neobsahuju
    #          interaktivne casti

## Prostredie LaTeX-u

Výsledkom konverzie je sada súborov **\*tex** v adresároch pomenovaných podľa notebookov. Štruktúra prostredia pre generovanie finálnych dokumentov má tvar (doporučené)

    <Moja_Kniha>
        |
        |     |
        |     +-- <meno_notebooku_01>        generovany adresar
        |     |     |
        |     |     +-- notebook_01.ipynb    kopia originalneho notebooku
        |     |     +-- notebook_01.tex
        |     |     |
        |     |     +-- <meno_notebboku_01>
        |     |            |
        |     |            +-- *.png         skriptom generovane obrazky (UPDATE)
        |
        +-- <img>              kopia adresaru so statickymi (negenerovanymi)
        |      |               obrazkami pouzitymi v notebookoch
        |      +-- *.png       
        | 
        +--- main.tex          konfiguracia dokumenu, usporiadanie a poradie kapitol 
        +--- listing.tex       format zobrazenia vypisov programov
        +--- title_page.tex    titulna stranka dokumentu
        +--- chapters_path.tex cesty ku kapitolam  (needitovat !)
        +--- graphics_path.tex cestu k obrazkom (needitovat !)


V každom vygenerovanom adresári je podadresár vytváraný pri UPDATE notebooku, ktorý obsahuje súbory s obrázkami vygenerovanými skriptami notebookov. Pre spracovanie v systéme LaTeX-u treba jednotlivé súbory usporiadať do finálneho dokumentu a nastaviť cesty ku generovaným a vkladaným obrázkom, príslušné konfiguračné súbory vytvára skript automaticky.

    % konfiguracia dokumentu, pouzite prostredia, slovencina
    
    \documentclass[10pt,a4paper]{report}
    \usepackage[utf8]{inputenc}
    \usepackage[slovak]{babel}
    \usepackage[IL2]{fontenc}   
    ...
    
    %importovanie konfiguracneho suboru pre formatovanie vypisu programov    
    \include{listing}             

    % cesty k obrazkom
    \include{graphics_path} 
            
    % struktura dokumentu
    \begin{document}

    % titulna stranka
    \include{title_page}

    % obsah
    \newpage
    \tableofcontents

    % kapitoly dokumentuy
    \include{chapters_path}
    
    \end{document}

Výsledný dokument vytvárame obvyklým spôsobom, z prostredia editora LaTeX-u alebo skriptom.


## Čo (zatiaľ) nefunguje ... a doporučenia

V automatickom móde generovania LaTeXu zatiaľ nefunguje

* Matematické výrazy v popise obrázkov
* Tabulky - je v tom zatiaľ chaos ... asi vhodnejšie je tabulky vytvoriť vo vlastnom *\*.tex* súbore a vložiť ich do finálneho dokumentu cez *include*
* Odkazy medzi medzi kapitolami na vzorce a pod.
* Variabilné nastavovanie veľkosti generovaných obrázkov, nastavenie veľkosti pomocou *figsize=()* sa neprenáša do LaTeXu
* Použitie italic textu v nadpisoch generuje chybu

Parser Jupyter-u sa v niektorých prípadoch mierne odlišuje od LaTeX-u, prípadne je tolerantnejší k nedodržaniu pravidiel LaTeX-u. Pri tvorbe notebookov je vhodné dodržiavať niektoré odporučenia

* Vkladané obrázky v \*.png formáte generovať v jednom rozlíšeni (napr. 600dpi)
* Pri vkladaní matematických výrazov do textu nevkladať medzery za a pred znakom koncového znaku dolára.

